In [11]:
import pandas as pd
data=pd.read_csv('census_data.csv')
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [12]:
data['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [0]:
def label_fix(label):
  if label==' <=50K':
    return 0
  else:
    return 1
 

In [14]:
data['income_bracket']=data['income_bracket'].apply(label_fix)
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [0]:
X_data=data.drop('income_bracket',axis=1)
Y_data=data['income_bracket']

In [0]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test=train_test_split (X_data,Y_data,test_size=0.3,random_state=101)

In [17]:
data.columns


Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [0]:
import tensorflow as tf
gender=tf.feature_column.categorical_column_with_vocabulary_list("gender",["Female","Male"])
occupation=tf.feature_column.categorical_column_with_hash_bucket("occupation",hash_bucket_size=1000)
marital_status=tf.feature_column.categorical_column_with_hash_bucket("marital_status",hash_bucket_size=1000)
relationship=tf.feature_column.categorical_column_with_hash_bucket("relationship",hash_bucket_size=1000)
education=tf.feature_column.categorical_column_with_hash_bucket("education",hash_bucket_size=1000)
workclass=tf.feature_column.categorical_column_with_hash_bucket("workclass",hash_bucket_size=1000)
native_country=tf.feature_column.categorical_column_with_hash_bucket("native_country",hash_bucket_size=1000)
age=tf.feature_column.numeric_column("age")
education_num=tf.feature_column.numeric_column("education_num")
capital_gain=tf.feature_column.numeric_column("capital_gain")
capital_loss=tf.feature_column.numeric_column("capital_loss")
hours_per_week=tf.feature_column.numeric_column("hours_per_week")
feat_cols=[gender,occupation,marital_status,relationship,education,workclass,native_country,age,education_num,capital_gain,capital_loss,hours_per_week]



In [0]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=Y_train,batch_size=100,num_epochs=None,shuffle=True)


In [24]:
model=tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpd0ee6s5e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd7d9dcdd68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
model.train(input_fn=input_func,steps=10000)

In [0]:
pred_fn=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [0]:
pred_gen=model.predict(input_fn=pred_fn)

In [0]:
predictions=list(pred_gen)

In [0]:
final_preds=[pred['class_ids'][0] for pred in predictions]


In [35]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,final_preds))


             precision    recall  f1-score   support

          0       0.89      0.92      0.90      7436
          1       0.71      0.63      0.67      2333

avg / total       0.84      0.85      0.85      9769

